# Imports

In [1]:
import os
import datetime as dt
import requests
from datetime import datetime

import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import random


# Inputs

In [2]:
def build_cookie(file):
    keys = ['_ga','session','_gid','_gat']
    cookie = ''

    with open(file) as f:
        for l in f:
            l = l.replace('\n','')
            l = l.split('\t')
            if any(x in l for x in keys):
                cookie += l[-2] + '=' + l[-1] + '; '

    return cookie[:-2]

In [3]:
def get_data(id_number: int, cookie: str, year: int = 2020):
    """Retrieves AoC Leaderboard data as JSON (i.e., Dict)

    Args:
        id_number: int - identification number for leaderboard info;
            viewable in the URL for downloading the info manually
        year: int - year for which to retrieve info
        cookie: str - cookie information with which to retrieve info;
            viewable in developer tools upon download (~1 month expiration)

    Returns:
        AoC Leaderboard information as JSON

    ```
    import leaderboard
    aoc_leaderboard_info = leaderboard.get_data(1524967)
    ```
    """
    result = requests.get(
        f'https://adventofcode.com/{year}/leaderboard/private/view/{id_number}.json',
        headers={'cookie': cookie},
    )
    return result.json()

In [4]:
curr_dir = os.curdir

In [29]:
# get cookie
cookie = build_cookie('adventofcode.com_cookies-jb.txt')

# read in leaderboard data
data = get_data(1542191,cookie,2020)

# read in contestant data
file = r'Contestants.xlsx'
contestants = pd.read_excel(os.path.join(curr_dir, file),dtype=str)

# read in deadline data
file = r'deadlines.xlsx'
deadlines = pd.read_excel(os.path.join(curr_dir, file))

# read in past winners
# file = r'winners.xlsx'
# winners = pd.read_excel(os.path.join(curr_dir,file))

# Parse data

In [30]:
def parse_json(data, contestants):
    """parse the AoC leaderboard data into a dataframe with one row per participant
    data: leaderboard data from JSON file
    contestants: dataframe with contestant names and AoC display names
    returns: df dataframe with one row per contestant and submission times per problem/part"""

    df = pd.DataFrame()

    for m in data['members'].keys():
        name = data['members'][m]['name']
        name = m if name is None else name
        if name in contestants['AoC Display name'].values:
            print_name = contestants[contestants['AoC Display name'] == name].Name.values[0]
            times = {'Contestant': print_name, 'score': int(data['members'][m]['local_score']),
                     'stars': int(data['members'][m]['stars'])}
            for day in data['members'][m]['completion_day_level'].keys():
                for p in data['members'][m]['completion_day_level'][day].keys():
                    times[day + '_' + p + '_time'] = dt.datetime.fromtimestamp(
                        data['members'][m]['completion_day_level'][day][p]['get_star_ts'])

            df = df.append(times, ignore_index=True)

    for col in df.columns:
        if col in ['score', 'stars']:
            df[col] = df[col].astype(int)
        elif col == 'Contestant':
            pass
        else:
            problem = col.replace('time', 'rank')
            df[problem] = df[col].rank()

    df.sort_values(by='Contestant', inplace=True)
    df.reset_index(inplace=True, drop=True)

    return df


In [39]:
results = parse_json(data, contestants)

In [40]:
results = results.drop([x for x in results.columns if ("_1" in x) or ("rank" in x)] + ['score', 'stars'], axis=1, errors='ignore')

In [41]:
puzzle_order = {"1": 1, "6": 2, "5": 3, "4":4, "7":5}

In [44]:
results.columns = [x.replace("_2", "") for x in results.columns]
results = results.drop(["2_time", "3_time"],axis=1)

In [46]:
new_cols = []
for col_name in results.columns:
    for key in puzzle_order.keys():
        col_name = col_name.replace(key, str(puzzle_order[key]))
    new_cols.append(col_name)

In [47]:
results.columns = new_cols

# Do stuff

In [49]:
deadlines.reset_index(inplace=True)
deadlines['index'] = deadlines['index'] + 1

In [50]:
deadlines = deadlines.rename(columns={"index":"week"})

In [51]:
# Well this is brittle
results = pd.melt(results, id_vars=['Contestant'], value_vars=['2_time', '1_time', '3_time', '4_time', '5_time'])

In [52]:
results['variable'] = results.variable.str.replace("_time", "")

In [53]:
results = results.rename(columns={"variable":"week", "value":"submission_time"})

In [54]:
results.week = results.week.astype(int)
results['submission_time'] = pd.to_datetime(results['submission_time']).dt.date

In [55]:
submissions = results.merge(deadlines, how='left', on="week")
submissions['Deadline'] = pd.to_datetime(submissions['Deadline']).dt.date

In [56]:
submissions['days_from_deadline'] = submissions.Deadline - submissions.submission_time

In [57]:
submissions = submissions.sort_values('days_from_deadline', ascending=False)

In [58]:
submissions

,Contestant,week,submission_time,Problem,Deadline,days_from_deadline
33,Alex Adamczyk,1,2020-12-01,1,2022-02-22,448 days
123,Robert Morsch,4,2022-02-21,4,2022-03-14,21 days
103,Cassandra Pray,4,2022-02-24,4,2022-03-14,18 days
91,Robert Morsch,3,2022-02-21,5,2022-03-07,14 days
43,David Gregory,1,2022-02-14,1,2022-02-22,8 days
...,...,...,...,...,...,...
152,Mike McNerney,5,NaN,7,2022-03-21,NaT
155,Robert Morsch,5,NaN,7,2022-03-21,NaT
156,Sean Spencer,5,NaN,7,2022-03-21,NaT
158,Yasmine Sikder,5,NaN,7,2022-03-21,NaT


In [59]:
submissions['cum_count'] = submissions.groupby("week").cumcount()

In [60]:
submissions['days_from_deadline'] = submissions['days_from_deadline'].dt.days * -1

In [61]:
submissions

,Contestant,week,submission_time,Problem,Deadline,days_from_deadline,cum_count
33,Alex Adamczyk,1,2020-12-01,1,2022-02-22,-448.0,0
123,Robert Morsch,4,2022-02-21,4,2022-03-14,-21.0,0
103,Cassandra Pray,4,2022-02-24,4,2022-03-14,-18.0,1
91,Robert Morsch,3,2022-02-21,5,2022-03-07,-14.0,0
43,David Gregory,1,2022-02-14,1,2022-02-22,-8.0,1
...,...,...,...,...,...,...,...
152,Mike McNerney,5,NaT,7,2022-03-21,NaN,27
155,Robert Morsch,5,NaT,7,2022-03-21,NaN,28
156,Sean Spencer,5,NaT,7,2022-03-21,NaN,29
158,Yasmine Sikder,5,NaT,7,2022-03-21,NaN,30


In [62]:
for_viz = submissions[1:].groupby(["week","days_from_deadline"]).max("cum_count").reset_index()

In [63]:
import plotly.express as px
px.line(submissions[1:], y="cum_count", x="days_from_deadline", color="week")

In [64]:
only_submitted = submissions.dropna()

In [65]:
submissions.groupby(["week","submission_time"]).count()

Contestant  Problem  Deadline  days_from_deadline  \
week submission_time                                                      
1    2020-12-01                1        1         1                   1   
     2022-02-14               10       10        10                  10   
     2022-02-15                5        5         5                   5   
     2022-02-16                3        3         3                   3   
     2022-02-17                3        3         3                   3   
     2022-02-18                1        1         1                   1   
     2022-02-21                1        1         1                   1   
     2022-02-24                1        1         1                   1   
     2022-02-27                1        1         1                   1   
2    2022-02-21                6        6         6                   6   
     2022-02-22                5        5         5                   5   
     2022-02-23                2        2         2                   2   
     2022-02-24                2        2         2                   2   
     2022-02-26                2        2         2                   2   
     2022-02-27                4        4         4                   4   
     2022-03-01                1        1         1                   1   
     2022-03-09                1        1         1                   1   
     2022-03-13                1        1         1                   1   
3    2022-02-21                1        1         1                   1   
     2022-02-28                8        8         8                   8   
     2022-03-01                2        2         2                   2   
     2022-03-02                1        1         1                   1   
     2022-03-04                2        2         2                   2   
     2022-03-05                1        1         1                   1   
     2022-03-06                2        2         2                   2   
     2022-03-08                1        1         1                   1   
4    2022-02-21                1        1         1                   1   
     2022-02-24                1        1         1                   1   
     2022-03-07                5        5         5                   5   
     2022-03-08                1        1         1                   1   
     2022-03-09                4        4         4                   4   
     2022-03-10                1        1         1                   1   
     2022-03-11                1        1         1                   1   
     2022-03-12                1        1         1                   1   
     2022-03-13                3        3         3                   3   
     2022-03-14                1        1         1                   1   
     2022-03-15                1        1         1                   1   
     2022-03-20                1        1         1                   1   
5    2022-03-14                4        4         4                   4   
     2022-03-15                1        1         1                   1   
     2022-03-16                2        2         2                   2   
     2022-03-17                1        1         1                   1   
     2022-03-18                2        2         2                   2   
     2022-03-19                2        2         2                   2   
     2022-03-20                3        3         3                   3   

                      cum_count  
week submission_time             
1    2020-12-01               1  
     2022-02-14              10  
     2022-02-15               5  
     2022-02-16               3  
     2022-02-17               3  
     2022-02-18               1  
     2022-02-21               1  
     2022-02-24               1  
     2022-02-27               1  
2    2022-02-21               6  
     2022-02-22               5  
     2022-02-23               2  
     2022-02-24     

In [66]:
only_submitted.groupby("week").count().reset_index()[['week', 'Contestant']]

,week,Contestant
0,1,26
1,2,24
2,3,18
3,4,21
4,5,15
